# Invoke-WebRequest
**Objective**: Scrape data from a webpage without an API service

**Website**: [Quotes to Scrape](quotes.toscrape.com)
- Quote
- Author
- Tags

<img src="https://github.com/MergeWords/PowerShell/blob/Invoke-WebRequest/Invoke-WebRequest/Images/screencapture-quotes-toscrape-2020-06-12-02_16_10-edit.png?raw=true" width="100%">

## Step 1 - Inspect HTML structure

In [6]:
$WebSite = Invoke-WebRequest "http://quotes.toscrape.com/"
$WebSite



StatusCode        : 200
StatusDescription : OK
Content           : <!DOCTYPE html>
                    <html lang="en">
                    <head>
                    	<meta charset="UTF-8">
                    	<title>Quotes to Scrape</title>
                        <link rel="stylesheet" href="/static/bootstrap.min.css">
                        <link rel="stylesheet" href="/static/m...
RawContent        : HTTP/1.1 200 OK
                    Connection: keep-alive
                    X-Upstream: spidyquotes-master_web
                    Content-Length: 11053
                    Content-Type: text/html; charset=utf-8
                    Date: Fri, 12 Jun 2020 11:39:46 GMT
                    Server: nginx/1.14.0 (U...
Forms             : {}
Headers           : {[Connection, keep-alive], [X-Upstream, spidyquotes-master_web], [Content-Length, 11053], 
                    [Content-Type, text/html; charset=utf-8]...}
Images            : {}
InputFields       : {}
Links             : {@{i

## Step 2 - Use the **Content** method to view all the HTML for the site and pick out the elements needed to parse Quote, Author, and Tags data

In [7]:
$Website.Content

<!DOCTYPE html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Quotes to Scrape</title>
    <link rel="stylesheet" href="/static/bootstrap.min.css">
    <link rel="stylesheet" href="/static/main.css">
</head>
<body>
    <div class="container">
        <div class="row header-box">
            <div class="col-md-8">
                <h1>
                    <a href="/" style="text-decoration: none">Quotes to Scrape</a>
                </h1>
            </div>
            <div class="col-md-4">
                <p>
                
                    <a href="/login">Login</a>
                
                </p>
            </div>
        </div>
    

<div class="row">
    <div class="col-md-8">

    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">
        <span class="text" itemprop="text">"The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking."</span>
        <span>by <small class="author" itempr

- **Quote** is in the [_span_](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/span) element
- **Author** is in the [_small_](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/small) element
- **Tags** are in the [_meta_](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/meta) element